In [0]:
!pip install eli5

     |████████████████████████████████| 112kB 2.9MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import median_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

In [9]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [10]:
ls data

men_shoes.csv  shoes_prices.csv


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory= False)

In [0]:
#w tej funkcji model jest jako parametr zewnętrzy funkcji   
def run_model(feats,   model = DecisionTreeRegressor(max_depth = 5)):
  X = df[feats].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model,X, y, scoring = 'neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
df['brand_cat'] = df['brand'].factorize()[0]

In [16]:
run_model(['brand_cat'])

(-58.38655694633361, 4.223555478221712)

In [18]:
model = RandomForestRegressor(max_depth = 5, n_estimators=100, random_state=0)
run_model(['brand_cat'],model)

(-57.47223572384038, 4.328288468270897)

In [19]:
df.head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,reviews,sizes,skus,sourceurls,upc,vin,websiteids,weight,brand_cat
0,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",39.89,39.89,NaN,NaN,NaN,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,True,NaN,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
1,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",51.99,51.99,NaN,NaN,new,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,False,UnbeatableSale - Walmart.com,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
2,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,40.02,40.02,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2016-03-08T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
3,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,50.31,50.31,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2015-11-30T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
4,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""

In [0]:
#sprawdzmy brand z małej litery
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]

In [24]:
model = RandomForestRegressor(max_depth = 5, n_estimators=100, random_state=0)
run_model(['brand_cat'],model)

(-57.31783843165656, 4.181246596160967)

In [26]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [0]:
#ogarniemy ten slownik z gory
from ast import literal_eval
from tqdm import tqdm_notebook

In [32]:
#test 
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'
str_dict

'[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'

In [33]:
literal_eval(str_dict)

[{'key': 'Gender', 'value': ['Men']},
 {'key': 'Shoe Size', 'value': ['M']},
 {'key': 'Shoe Category', 'value': ["Men's Shoes"]},
 {'key': 'Color', 'value': ['Multicolor']},
 {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']},
 {'key': 'Brand', 'value': ['Josmo']}]

In [34]:
literal_eval(str_dict)[0]

{'key': 'Gender', 'value': ['Men']}

In [39]:
literal_eval(str_dict)[0]['value'][0]

'Men'

In [42]:
#pass puszcza pusta funkcje

def parse_features(x):
    if str(x) == 'nan' :return []
    return literal_eval(x)

df['features_parsed']= df['features'].map(parse_features)
#nie zadzialalo bo w >value":["17\\""]},{"key< jakis problem, dodajemy replace (nizej)

SyntaxError: ignored

In [0]:
def parse_features(x):
    if str(x) == 'nan' :return []
    return literal_eval(x.replace('\\"','"'))

df['features_parsed']= df['features'].map(parse_features)


In [48]:
df.features_parsed.head().values

array([list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Shoe Size', 'value': ['M']}, {'key': 'Shoe Category', 'value': ["Men's Shoes"]}, {'key': 'Color', 'value': ['Multicolor']}, {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']}, {'key': 'Brand', 'value': ['Josmo']}]),
       list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Shoe Size', 'value': ['M']}, {'key': 'Shoe Category', 'value': ["Men's Shoes"]}, {'key': 'Color', 'value': ['Multicolor']}, {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']}, {'key': 'Brand', 'value': ['Josmo']}]),
       list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Color', 'value': ['Black']}, {'key': 'Shipping Weight (in pounds)', 'value': ['0.45']}, {'key': 'Condition', 'value': ['New']}, {'key': 'Brand', 'value': ['SERVUS BY HONEYWELL']}, {'key': 'manufacturer_part_number', 'value': ['ZSR101BLMLG']}]),
       list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Color', 'value': ['Black']}, {'key': 'Shipping Weight (in

In [0]:
#chcemy bardziej uprosiccdo noralnego slownika


def parse_features(x):
  output_dict = {}
  if str(x) == 'nan' :return output_dict
  features = literal_eval(x.replace('\\"','"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()
    output_dict[key] = value
  return output_dict

df['features_parsed']= df['features'].map(parse_features)


In [53]:
df.features_parsed.head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [56]:
keys = set()

df['features_parsed'].map(lambda x: keys.update(x.keys()))

len(keys)

476

In [58]:
def get_name_feat (key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] =  df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [59]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_band type', 'feat_location - country',
       'feat_cleaning, care & maintenance', 'feat_seller inventory number',
       'feat_leg opening:', 'feat_has mercury', 'feat_transactionid',
       'feat_season', 'feat_product id', 'feat_height'],
      dtype='object', length=526)

In [63]:
df[False==df['feat_athlete'].isnull()].shape[0]/df.shape[0]*100
#przyklad, sprawdzamy jak czesto brakuje danej cechy (a teraz juz kolumny)

0.0437636761487965

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key]= df[False==df[get_name_feat (key)].isnull()].shape[0]/df.shape[0]*100

In [75]:
keys_stat

KeyError: ignored

In [73]:
{k:v for k,v in keys_stat.items() if v>30}  #tutaj nam pokazuje tecechy gdzie jest wiecej niz x%

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_genderd_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer_cat'] = df['feat_manufacturer'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]


In [83]:
df['brand'] = df['brand'].map(lambda x:str(x).lower())
df[df.brand != df.feat_brand].shape

(9434, 531)

In [0]:
feats=['']

In [84]:
model = RandomForestRegressor(max_depth = 5, n_estimators=100, random_state=0)
run_model(['brand_cat'],model)

(-57.31783843165656, 4.181246596160967)

In [86]:
feats = ['brand_cat','feat_brand_cat','feat_color_cat','feat_genderd_cat','feat_manufacturer_cat','feat_material_cat']
model = RandomForestRegressor(max_depth = 5, n_estimators=100, random_state=0)
run_model(feats,model)

(-57.137528728073804, 4.169243136850817)

In [89]:
X = df[feats].values
y=df['prices_amountmin'].values

m = RandomForestRegressor(max_depth = 5, n_estimators=100, random_state=0)
m.fit(X,y)

perm = PermutationImportance(m,random_state=1).fit(X,y);
eli5.show_weights(perm,feature_names=feats)

Weight,Feature
0.2578 ± 0.0112,brand_cat
0.1058 ± 0.0094,feat_material_cat
0.0423 ± 0.0043,feat_genderd_cat
0.0213 ± 0.0011,feat_brand_cat
0.0077 ± 0.0004,feat_color_cat
0.0002 ± 0.0000,feat_manufacturer_cat


In [91]:
feats = ['brand_cat','feat_brand_cat','feat_genderd_cat','feat_material_cat']
model = RandomForestRegressor(max_depth = 5, n_estimators=100, random_state=0)
run_model(feats,model)

(-57.20148875865986, 4.149725476371453)

In [0]:
#brand_cat jest wazny, przyjrzyjmy sie mu lepiej

In [95]:
df['brand'].value_counts(normalize=True)

nike              0.097210
puma              0.033315
ralph lauren      0.028775
vans              0.021116
new balance       0.020295
                    ...   
shoe goo          0.000055
totoro            0.000055
rugged blue       0.000055
solo, solo        0.000055
american eagle    0.000055
Name: brand, Length: 1732, dtype: float64

In [99]:
df[df['brand'] == 'nike'].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [103]:
feats = ['brand_cat','feat_brand_cat','feat_genderd_cat','feat_material_cat', 'feat_sport_cat', 'feat_style_cat']
model = RandomForestRegressor(max_depth = 5, n_estimators=100, random_state=0)
run_model(feats,model)

(-57.17319916647938, 4.240863641857993)

In [104]:
X = df[feats].values
y=df['prices_amountmin'].values

m = RandomForestRegressor(max_depth = 5, n_estimators=100, random_state=0)
m.fit(X,y)

perm = PermutationImportance(m,random_state=1).fit(X,y);
eli5.show_weights(perm,feature_names=feats)

Weight,Feature
0.2608 ± 0.0111,brand_cat
0.1046 ± 0.0083,feat_material_cat
0.0455 ± 0.0046,feat_genderd_cat
0.0216 ± 0.0012,feat_brand_cat
0.0070 ± 0.0013,feat_style_cat
0.0003 ± 0.0000,feat_sport_cat


In [105]:
df[df['brand'] == 'nike'].features_parsed.sample(5).values

array([{'style': 'athletic', 'condition': 'new with box'}, {},
       {'sport': 'soccer', 'type': 'cleats', 'color': 'yellow', 'name': 'mercurial vapor ix', 'condition': 'new without box', 'gender': 'men', 'main color': 'yellow'},
       {},
       {'gender': 'men', 'shoe category': 'mens shoes', 'color': 'black/beige', 'casual & dress shoe style': 'athletic sneakers', 'manufacturer part number': '366177018', 'brand': 'nike', 'shoe width': 'medium (d, m)'}],
      dtype=object)

In [0]:
for key in keys:
  df[get_name_feat (key) + '_cat'] = df[get_name_feat (key)].factorize()[0]


In [109]:
feats_cat = [x for x in df.columns if 'cat' in x]
feats_cat

['categories',
 'brand_cat',
 'feat_shoe category',
 'feat_multi pack indicator',
 'feat_recommended location',
 'feat_clothing category',
 'feat_certifications and listings',
 'feat_fabrication',
 'feat_catalog',
 'feat_location - city/state',
 'feat_location - country',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_genderd_cat',
 'feat_manufacturer_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_frame color_cat',
 'feat_characteristics_cat',
 'feat_theme_cat',
 'feat_full product manual_cat',
 'feat_shipping weight (in pounds)_cat',
 'feat_article_cat',
 'feat_resizable_cat',
 'feat_elastic banded waist_cat',
 'feat_material detail-1_cat',
 'feat_polarized_cat',
 'feat_is weather-resistant_cat',
 'feat_golf shoe type_cat',
 'feat_combined shipping_cat',
 'feat_case material_cat',
 'feat_interior pockets_cat',
 'feat_is polarized_cat',
 'feat_clothing product type_cat',
 'feat_bridge size:_cat',
 'feat_face shape_cat',
 'feat_age group_cat',
 'feat_lined_cat',


In [0]:
feats = ['brand_cat','feat_brand_cat','feat_genderd_cat','feat_material_cat', 'feat_sport_cat', 'feat_style_cat']
#feats += feats_cat
#robimy unikalna liste
#feats = list(set(feats))

In [114]:
X = df[feats].values
y=df['prices_amountmin'].values

m = RandomForestRegressor(max_depth = 5, n_estimators=100, random_state=0)
m.fit(X,y)

perm = PermutationImportance(m,random_state=1).fit(X,y);
eli5.show_weights(perm,feature_names=feats)

Weight,Feature
0.2608 ± 0.0111,brand_cat
0.1046 ± 0.0083,feat_material_cat
0.0455 ± 0.0046,feat_genderd_cat
0.0216 ± 0.0012,feat_brand_cat
0.0070 ± 0.0013,feat_style_cat
0.0003 ± 0.0000,feat_sport_cat


In [113]:
model = RandomForestRegressor(max_depth = 5, n_estimators=100, random_state=0)
run_model(feats,model)

(-57.17319916647938, 4.240863641857993)

In [0]:
feats = ['feat_ring size_cat','feat_movement_cat','feat_metal type_cat','brand_cat','feat_brand_cat','feat_genderd_cat','feat_material_cat', 'feat_sport_cat', 'feat_style_cat']
#feats += [ ]


In [136]:
model = RandomForestRegressor(max_depth = 5, n_estimators=100, random_state=0)
result = run_model(feats,model)
run_model(feats,model)

(-57.32286299244625, 4.188786879962681)

In [137]:
X = df[feats].values
y=df['prices_amountmin'].values

m = RandomForestRegressor(max_depth = 5, n_estimators=100, random_state=0)
m.fit(X,y)
print(result)
perm = PermutationImportance(m,random_state=1).fit(X,y);
eli5.show_weights(perm,feature_names=feats)

(-57.32286299244625, 4.188786879962681)


Weight,Feature
0.2547 ± 0.0085,brand_cat
0.1037 ± 0.0068,feat_material_cat
0.0235 ± 0.0028,feat_genderd_cat
0.0182 ± 0.0013,feat_brand_cat
0.0160 ± 0.0008,feat_movement_cat
0.0067 ± 0.0012,feat_metal type_cat
0.0034 ± 0.0004,feat_ring size_cat
0.0031 ± 0.0005,feat_style_cat
0.0002 ± 0.0000,feat_sport_cat


In [132]:
df['weight'].unique()

array([nan, '3.0 lbs', '9 g', '1.45 lbs', '0.45 lbs', '1.0 lbs',
       '0.23 lbs', '5.0 lbs', '5.5 lbs', '7.45 lbs', '4.0 lbs',
       '2.7969 lbs', '3.9 lbs', '4.6 pounds', '2.1 lbs', '1.1057 lbs',
       '15.0 lbs', '2.4 ounces', '454 g', '0.105 lbs', '9.1 ounces',
       '4.8 lbs', '6.1 lbs', '6.5 lbs', '1.1041 lbs', '1.3 Kg', '91 g',
       '20.0 lbs', '6.0 lbs', '386 g', '0.81 lbs', '4.5 lbs',
       '0.5 ounces', '2.0 lbs', '3.13 lbs', '5.9 lbs', '6.15 lbs',
       '1 pounds', '1.95 lbs', '2.15 lbs', '2 pounds', '2.1 pounds',
       '14 Kg', '0.4788 lbs', '10.0 lbs', '0.38 lbs', '2.5 lbs',
       '68.912 lbs', '45 g', '13.09 lbs', '2.5 pounds', '0.21 lbs',
       '16.75 lbs', '6.3 lbs', '272 g', '1.8 Kg', '2.8 pounds', '0.1 lbs',
       '5.05 lbs', '0.28 lbs', '76.08 lbs', '0.15 lbs', '200 g',
       '7.8 pounds', '399 g', '4.95 lbs', '64.144 lbs', '24 pounds',
       '73.696 lbs', '1.6 lbs', '6.6 ounces', '5 g', '1.2 Kg', '862 g',
       '3.05 lb', '8.6 ounces', '3.6 lbs', '71.